設定ファイルを読み込む

- ``sandbox/config.toml``
- 質問をカテゴリー型に変換するための設定
- ``[questions]``: 質問の内容を取得するための設定
- ``[choices]``: 質問の回答をカテゴリー型に変換するための設定
- ``choices``は使いまわすことがあるので、``questions``と1対1に対応させず、別々のセクションにした
  - 例: ``regional`` / ``subregional`` / ``good_poor`` / ``agree_disgree`` など

In [ ]:
from pathlib import Path
import pandas as pd
import tomllib

``tomllib``を使って設定ファイルを読み込む

In [ ]:
f_cfg = "../sandbox/config.toml"
fname = Path(f_cfg)
with fname.open("rb") as f:
    config = tomllib.load(f)

キーを確認する

In [ ]:
config.keys()

In [ ]:
nd = config.get("defined", "Not defined")
volumes = config.get("volumes", "Not Defined")
questions = config.get("questions", "Not Defined")
choices = config.get("choices", "Not Defined")
o = config.get("options", "Not Defined")
options = pd.DataFrame(o)

In [ ]:
nd

In [ ]:
volumes.keys()

In [ ]:
questions.keys()

In [ ]:
choices.keys()

質問の概要オプションをデータフレームに変換する

In [ ]:
options["name"].tolist()
options.query("type == 'categorical'")["name"].to_list()
options.query("type == 'categorical'")["description"].to_list()
options.query("type == 'categorical'")[["name", "title"]].to_dict().keys()

## ``config``モジュールの動作確認

In [ ]:
import titanite as ti
f_cfg = "../sandbox/config.toml"
c = ti.Config(load_from=f_cfg)
# c.load_categories()
# c.load_options()
c.load()
#categories = c.categorical()

``self.config``の値を確認する

In [ ]:
#c.config.get("volumes", "Not defined")
#c.config.get("questions", "Not defined")
#c.config.get("choices", "Not defined").items()
#c.config.get("options", "Not defined")
#c.config

- ``categorical()`` を ``load_categories()``で置き換える
- ``self.categories``でアクセスできるようにした

In [ ]:
c.categorical() == c.categories

- 質問に関係するオプション一式はデータフレームにした
- ``load_options()``で読み込み＆変換ができる
- ``self.options``でアクセスできるようにした

In [ ]:
c.options